In [ ]:
import os

# create a large OMAP entry, deep-scrub it, then try to kill the Primary in
# such a way that the info change noted by the scrub is never propagated
# to the new primary

# List the current directory
#print(os.listdir(.))

In [ ]:
# Ask the user to select a digit in the range 0 to 8
k = int(input("Please select a digit in the range 0 to 8: "))

# Check if the digit is within the valid range
if 0 <= k <= 8:
    print(f"will be running in k{k}")
else:
    print("Invalid selection. Please select a digit between 0 and 8.")
    exit(1)

from pathlib import Path

wd = Path(f"/home/rfriedma/src/k{k}/ceph/build")
os.chdir(wd)
%env CEPH_JTEST_ROOT=/home/rfriedma/src/k{k}/ceph/build
!echo $CEPH_JTEST_ROOT > /tmp/jpath


In [ ]:
!ls -l
!bash -c MGR=0 ls -l


In [ ]:
%%bash


function get_pg() {
    local poolname=$1
    local objectname=$2
    bin/ceph --format json osd map $poolname $objectname 2>/dev/null | jq -r '.pgid'
}

function get_primary() {
    local poolname=$1
    local objectname=$2

    bin/ceph --format json osd map $poolname $objectname 2>/dev/null | \
        jq '.acting_primary'
}




scrtch=to_"`date +%d_%H%M`"
echo $scrtch

MDS=0 MGR=1 OSD=5 MON=1 ../src/vstart.sh -n  --without-dashboard --msgr2 -X --memstore -o "memstore_device_bytes=68435456" -o "osd_op_queue=wpq"
sleep 2
bin/ceph -s

#bin/ceph tell osd.* config set debug_osd 20/20

bin/ceph config set global osd_pool_default_pg_autoscale_mode off
sleep 2

# disable rescheduling of the queue due to 'no-scrub' flags
bin/ceph tell osd.* config set osd_scrub_backoff_ratio 0.9999

# initial set of global scrub scheduling parameters
bin/ceph tell osd.* config set osd_scrub_interval_randomize_ratio 0.1
bin/ceph tell osd.* config set osd_deep_scrub_randomize_ratio 0
bin/ceph tell osd.* config set osd_scrub_min_interval 10
bin/ceph tell osd.* config set osd_scrub_max_interval 2000
bin/ceph tell osd.* config set osd_deep_scrub_interval 600


#PL1 is of size 2

bin/ceph osd pool create pl1 1 1
#bin/ceph osd pool autoscale-status
sleep 1
bin/ceph tell osd.* config set debug_osd 20/20
pl1_num=`bin/ceph osd pool stats pl1 | sed -n -r -e 's/.*id[^0-9]*([0-9]+)$/\1/p'`
echo $pl1_num
bin/ceph osd pool set pl1 size 2
bin/ceph osd pool set pl1 min_size 2
bin/ceph osd pool set pl1 pg_autoscale_mode off
bin/ceph osd pool stats
bin/ceph osd pool set pl1 noscrub 1
bin/ceph osd pool set pl1 nodeep-scrub 1
sleep 2

# create a new object with omap header and a >1MB OMAP entry
T=$(mktemp -p /tmp)
dd if=/dev/zero of="$T" bs=1024 count=1
bin/rados --pool pl1 put obj1 "$T" || return 1
dd if=/dev/zero of="$T" bs=1024 count=1025
bin/rados --pool pl1 setomapheader obj1 hdr-obj1 || return 1
bin/rados --pool pl1 setomapval obj1 key1 --input-file="$T" || return 1

pg=$(get_pg pl1 obj1)
primary=$(get_primary pl1 obj1)
echo "pg=$pg primary=$primary"


# set a small-enough OMAP entry limit
bin/ceph tell osd.* config set osd_deep_scrub_large_omap_object_value_sum_threshold 1024

# add enough objects to allow us time to kill the primary after detecting
# the omap issue. TBD

bin/ceph health detail
# deep-scrub to detect the large OMAP entry
bin/ceph tell $pg deep-scrub
sleep 2
bin/ceph health detail | grep omap





In [ ]:
%%bash
echo '-------->' $CEPH_JTEST_ROOT
jp=$CEPH_JTEST_ROOT
if [ -z $CEPH_JTEST_ROOT ]; then
    echo "CEPH_JTEST_ROOT is not set"
    [[ -f /tmp/jpath ]] && jp=`cat /tmp/jpath` || jp='.'
    echo '-------->' $jp
    %env CEPH_JTEST_ROOT=$jp
fi
cd $jp


In [ ]:
%%bash

file_path="/tmp/jpath"
if [ -f "$file_path" ]; then
        file_contents=$(cat "$file_path")
        echo "File contents read into variable."
else
        echo "File does not exist."
fi

In [ ]:
%%bash

#cd $CEPH_JTEST_ROOT

bin/ceph tell osd.0 dump_scrubs --format=json-pretty > /tmp/ds_00_b4params.json
bin/ceph tell osd.1 dump_scrubs --format=json-pretty > /tmp/ds_01_b4params.json
bin/ceph tell osd.2 dump_scrubs --format=json-pretty > /tmp/ds_02_b4params.json


# set the scheduling parameters

bin/ceph osd pool set pl1 scrub_min_interval 1
bin/ceph osd pool set pl1 scrub_max_interval 2
bin/ceph osd pool set pl1 deep_scrub_interval 3

#bin/ceph osd pool set pl2 scrub_min_interval 2
#bin/ceph osd pool set pl2 scrub_max_interval 4
#bin/ceph osd pool set pl2 deep_scrub_interval 10

sleep 3

bin/ceph tell osd.0 dump_scrubs --format=json-pretty > /tmp/ds_00.json
bin/ceph tell osd.1 dump_scrubs --format=json-pretty > /tmp/ds_01.json
bin/ceph tell osd.2 dump_scrubs --format=json-pretty > /tmp/ds_02.json





In [ ]:
%%bash

#%cd $CEPH_JTEST_ROOT

# common scrub configs
bin/ceph tell osd.* config set osd_blocked_scrub_grace_period 20
bin/ceph tell osd.* config set osd_stats_update_period_scrubbing 2
bin/ceph tell osd.* config set osd_stats_update_period_not_scrubbing 3
bin/ceph tell osd.* config set osd_scrub_backoff_ratio 0.9999
#bin/ceph tell osd.* config set osd_scrub_interval_randomize_ratio 0.1
#bin/ceph tell osd.* config set osd_deep_scrub_randomize_ratio 0

#bin/ceph tell mgr.$(bin/ceph mgr services | jq -r .mgr) config set mgr_stats_period 2


In [ ]:
%%bash

# list the scrub queue
scrtch=to_"`date +'%H%M%S'`"
echo $scrtch
bin/ceph tell osd.0 dump_scrubs --format=json-pretty
bin/ceph tell osd.1 dump_scrubs --format=json-pretty
bin/ceph tell osd.2 dump_scrubs --format=json-pretty
bin/ceph tell osd.0 dump_scrubs --format=json-pretty > /tmp/ds_0$scrtch.json
bin/ceph tell osd.1 dump_scrubs --format=json-pretty > /tmp/ds_1$scrtch.json
bin/ceph tell osd.2 dump_scrubs --format=json-pretty > /tmp/ds_2$scrtch.json


In [ ]:
%%bash
# set scrub parameters to guarantee slow scrub
bin/ceph tell osd.* config set osd_scrub_sleep "3.0"
bin/ceph tell osd.* config set osd_max_scrubs 1
bin/ceph tell osd.* config set osd_scrub_chunk_max 5
bin/ceph tell osd.* config set osd_shallow_scrub_chunk_max 5


In [ ]:
%%bash

# set higher urgency to one of the PGs
bin/ceph tell $pl1_num.7 scrub
bin/ceph tell $pl1_num.6 schedule-deep-scrub
sleep 1
bin/ceph pg dump pgs


In [ ]:
%%bash

scrtch=to_"`date +'%H%M%S'`"
echo $scrtch

# list the scrub queue
bin/ceph tell osd.0 dump_scrubs --format=json-pretty > /tmp/ds_0$scrtch.json
bin/ceph tell osd.1 dump_scrubs --format=json-pretty > /tmp/ds_1$scrtch.json
bin/ceph tell osd.2 dump_scrubs --format=json-pretty > /tmp/ds_2$scrtch.json
bin/ceph tell osd.0 dump_scrubs --format=json-pretty
bin/ceph tell osd.1 dump_scrubs --format=json-pretty
bin/ceph tell osd.2 dump_scrubs --format=json-pretty


In [ ]:
raise SystemExit("Stop here")

In [ ]:
%%bash

#cat /tmp/ds_01.json | jq -s '[.[]|.[]|select(.eligible==true)]| sort_by(.overdue,.pgid)' |  head -20

#cat /tmp/ds_01.json | jq -s '[.[]|.[]|select(.eligible==true)|  { pgid, overdue, sched_time } ]' |  head -20

#cat /tmp/ds_01.json | jq -s '[.[]|.[]|select(.eligible==true)|  { pgid, overdue, sched_time } ]' |  head -20

#cat ds_01.json | jq -s '[.[]|.[]|select(.eligible==false) | .overdue as $ov | . += { "ov":$ov } | [.] ] '|  head -20

#cat ds_01.json | jq -s '[.[]|.[]|select(.eligible==false) | .overdue as $ov | . += { "ov":$ov|not } | [.] ] '|  head -20

cat /tmp/ds_01.json | jq -s '[.[]|.[]|select(.eligible==true) | .overdue as $ov | . += { "ov":$ov|not } ]| sort_by(.ov,.sched_time,.pgid) | [.]  '|  head -20

#cat /tmp/ds_01.json | jq -s '[.[]|.[]|select(.eligible==true) | .overdue as $ov | .level as $lvl | . += { "ov":$ov|not, "lvl":$lvl } ]| sort_by(.ov,.sched_time,.pgid,.lvl) | [.]  '|  head -20

cat /tmp/ds_01.json | jq -s '[.[]|.[]|select(.eligible==true) | .overdue as $ov | .level as $lvl |
 . += { "ov":$ov|not, "lvl":$lvl } ]| sort_by(.ov,.sched_time,.pgid,.lvl) | [.]  '|  head -20

# use 'eligible' as just one more sort criteria
echo "========================== --- "
cat /tmp/ds_01.json | jq -s '[.[]|.[]| .eligible as $ripe | .overdue as $ov | .level as $lvl |
 . += { "not_ripe":$ripe|not, "not_ov":$ov|not, "lvl":$lvl } ] | 
 sort_by(.not_ripe, .not_ov,.sched_time,.pgid,.lvl) | [.]  '|  head -100

# now - make the sort an irregular one: if comparing the the targets of one PG, level tramps sched time

cat /tmp/ds_01.json | jq -s '
def inter_pg:
  if 
[.[]|.[]| 


# Termination


In [ ]:
%%bash
echo '-------->' $CEPH_JTEST_ROOT
cd $CEPH_JTEST_ROOT
if [ -z $CEPH_JTEST_ROOT ]; then
    echo "CEPH_JTEST_ROOT is not set"
    [[ -f /tmp/jpath ]] && jp=`cat /tmp/jpath` || jp='.'
    echo '-------->' $jp
    cd $jp
    %env CEPH_JTEST_ROOT=$jp
fi

pwd

../src/stop.sh
sleep 4
../src/stop.sh
